In [1]:
!which python

/bin/python


In [2]:
import torch
torch.cuda.is_available()

True

In [1]:
!pip install matplotlib

In [2]:
import matplotlib.pyplot as plt

In [3]:
from ultralytics import YOLO



In [ ]:

rs_pipeline = rs.pipeline()
rs_config = rs.config()
rs_config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
rs_config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)

In [3]:
import cv2
import numpy as np
import pyrealsense2 as rs
import ctypes
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8x.engine", task="segment")
# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:
        # Wait for a new frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        # Convert RealSense frame to OpenCV format
        color_image = np.asanyarray(color_frame.get_data())
        res = model.predict(color_image, imgsz=(736,1280))

        # Perform object detection using YOLOv8
        # Replace this with your actual object detection code using the YOLOv8 model
        
        # Display the results
        cv2.imshow("Object Detection", res[0].plot())
        
        # Check for the 'q' key to quit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()

# Close all OpenCV windows
cv2.destroyAllWindows()


Loading yolov8x.engine for TensorRT inference...
[04/29/2024-02:57:22] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.

[04/29/2024-02:57:22] [TRT] [I] Loaded engine size: 275 MiB
[04/29/2024-02:57:22] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +33, now: CPU 2132, GPU 23693 (MiB)
[04/29/2024-02:57:22] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +1, GPU +274, now: CPU 1, GPU 423 (MiB)
[04/29/2024-02:57:22] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +27, now: CPU 1857, GPU 23430 (MiB)
[04/29/2024-02:57:22] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +330, now: CPU 1, GPU 753 (MiB)

0: 736x1280 (no detections), 523.6ms
Speed: 9.5ms preprocess, 523.6ms inference, 9.0ms postprocess per image at shape (1, 

In [3]:
!pip list | grep torch

torch                                2.1.0a0+41361538.nv23.6
torchvision                          0.16.1                 


In [1]:
import cv2
import numpy as np
import pyrealsense2 as rs
import ctypes
from ultralytics import YOLO
import queue
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import cycle

# Initialize queues
input_queue = queue.Queue()
output_queue = queue.Queue()

# Load YOLOv8 models in separate instances
def load_model():
    return YOLO("yolov8n.engine", task="segment")

# Number of model instances
num_models = 4

# Create a pool of YOLO models
model_pool = [load_model() for _ in range(num_models)]

# Prediction function to be run by the workers
def process_image(data):
    color_image, model = data
    result = model.predict(color_image, imgsz=(736, 1280))
    return result[0].plot()  # Assuming result[0].plot() returns the image with predictions drawn

def worker(input_queue, output_queue, model_pool):
    with ThreadPoolExecutor(max_workers=len(model_pool)) as executor:
        model_cycle = cycle(model_pool)  # Create a cycle of models

        while True:
            if input_queue.empty():
                continue  # Wait until the queue has an item

            color_image = input_queue.get()  # Fetch the next image to process
            if color_image is None:
                break  # Stop if None is sent as a signal to terminate

            # Get the next model from the cycle
            model = next(model_cycle)
            
            # Submit the task to the executor
            future = executor.submit(process_image, (color_image, model))
            
            # Process the result as soon as it's ready
            result = future.result()
            output_queue.put(result)

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Main execution flow in the try block
try:
    # Start the worker thread
    from threading import Thread
    worker_thread = Thread(target=worker, args=(input_queue, output_queue, model_pool))
    worker_thread.start()

    while True:
        # Wait for a new frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        # Convert RealSense frame to OpenCV format and add to the input queue
        color_image = np.asanyarray(color_frame.get_data())
        input_queue.put(color_image)

        # Display results if available in the output queue
        if not output_queue.empty():
            result = output_queue.get()  # Get the next processed image
            cv2.imshow("Object Detection", result)

        # Check for the 'q' key to quit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Cleanup
    input_queue.put(None)
    worker_thread.join()
    pipeline.stop()
    cv2.destroyAllWindows()

Loading yolov8n.engine for TensorRT inference...
[04/29/2024-02:54:05] [TRT] [I] Loaded engine size: 13 MiB
[04/29/2024-02:54:06] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +616, GPU +870, now: CPU 1579, GPU 24717 (MiB)
[04/29/2024-02:54:06] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +13, now: CPU 0, GPU 13 (MiB)
[04/29/2024-02:54:06] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +28, now: CPU 1566, GPU 24718 (MiB)
[04/29/2024-02:54:06] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +61, now: CPU 0, GPU 74 (MiB)

0: 736x1280 1 WB, 39.7ms
Speed: 7.6ms preprocess, 39.7ms inference, 443.6ms postprocess per image at shape (1, 3, 736, 1280)
Loading yolov8n.engine for TensorRT inference...
[04/29/2024-02:54:08] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfe

RuntimeError: Frame didn't arrive within 5000